In [ ]:
!pip install transformers

import gradio as gr
from PyPDF2 import PdfReader
from langchain.vectorstores import FAISS
from langchain.text_splitter import RecursiveCharacterTextSplitter
from sentence_transformers import SentenceTransformer
from transformers import AutoTokenizer, AutoModelForCausalLM


In [ ]:
embedding_model = SentenceTransformer('all-MiniLM-L6-v2')
tokenizer = AutoTokenizer.from_pretrained("tiiuae/falcon-7b-instruct")
chat_model = AutoModelForCausalLM.from_pretrained("tiiuae/falcon-7b-instruct", device_map="auto")

In [ ]:
def interface_logic(task, files, question):
    response = ""
    if task == "Process PDFs":
        text = ""

        if files:
            for file in files:
                pdf_reader = PdfReader(file)
                for page in pdf_reader.pages:
                    text += page.extract_text()
            text_chunks = RecursiveCharacterTextSplitter(chunk_size=20000, chunk_overlap=2500).split_text(text)
            embeddings = [embedding_model.encode(chunk) for chunk in text_chunks]
            vector_store = FAISS.from_embeddings(embeddings, text_chunks)
            vector_store.save_local("faiss_index")
            response = "PDFs processed successfully!"
        else:
            response = "Please upload PDF files."
    elif task == "Ask Question":
        try:
            db = FAISS.load_local("faiss_index", embedding_model)
            docs = db.similarity_search(question)
            context = "\n".join([doc.page_content for doc in docs])
        except Exception as e:
            response = f"Error loading FAISS index or no context found: {e}"
            context = ""

        if context:
            input_text = f"Context: {context}\n\nQuestion: {question}"
            inputs = tokenizer(input_text, return_tensors="pt", truncation=True)
            outputs = chat_model.generate(**inputs, max_new_tokens=500, temperature=0.3)
            response = tokenizer.decode(outputs[0], skip_special_tokens=True).strip()
        else:
            response = "The answer is not available in the context."
    return response


In [6]:

interface = gr.Interface(
    fn=interface_logic,
    inputs=[
        gr.Radio(["Process PDFs", "Ask Question"], label="Select Task"),
        gr.File(file_types=[".pdf"], label="Upload PDF Files", file_count="multiple"),
        gr.Textbox(label="Ask a Question", placeholder="Type your question here"),
    ],
    outputs=gr.Textbox(label="Response")
)

interface.launch()


tokenizer_config.json:   0%|          | 0.00/1.13k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.73M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/281 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.05k [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/17.7k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.95G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/4.48G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/117 [00:00<?, ?B/s]

Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://6912b374dd979589e0.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
